In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('test_data.csv', sep=',')

In [3]:
df.columns

Index(['dlg_id', 'line_n', 'role', 'text'], dtype='object')

In [4]:
searchfor_greet = ['здравствуйте', 'добрый день']
filt_greet = pd.Series(df['text'].str.lower().str.contains('|'.join(searchfor_greet)))

In [5]:
searchfor_bye = ['до свидания']
filt_bye = pd.Series(df['text'].str.lower().str.contains('|'.join(searchfor_bye)))

In [6]:
df['greets'] = df.loc[filt_greet, ['text']]
df['goodbyes'] = df.loc[filt_bye, ['text']]

In [7]:
filt_notna_greet = (df['greets'].notna())
filt_notna_bye = (df['goodbyes'].notna())

filt_manager = (df['role'] == 'manager')

filt_greets_notna = filt_notna_greet | filt_notna_bye

In [8]:
df_greets = df.loc[(filt_greets_notna & filt_manager), ['dlg_id','role', 'greets', 'goodbyes']] 

In [9]:
df_greets

,dlg_id,role,greets,goodbyes
1,0,manager,Алло здравствуйте,NaN
108,0,manager,NaN,Всего хорошего до свидания
110,1,manager,Алло здравствуйте,NaN
163,1,manager,NaN,До свидания
166,2,manager,Алло здравствуйте,NaN
250,3,manager,Алло дмитрий добрый день,NaN
335,4,manager,NaN,Во вторник все ну с вами да тогда до вторника ...
479,5,manager,NaN,Ну до свидания хорошего вечера


In [10]:
df_greets_grouped = df_greets.groupby(['dlg_id']).count()

In [11]:
df_greets_grouped

,role,greets,goodbyes
dlg_id,,,
0,2,1,1
1,2,1,1
2,1,1,0
3,1,1,0
4,1,0,1
5,1,0,1


In [12]:
validated = df_greets_grouped.groupby(['dlg_id']).role.agg([lambda x: True if int(x) == 2 else False])

In [13]:
'Выполненное требование к менеджеру'

'Выполненное требование к менеджеру'

In [14]:
validated.columns = ['done instruction']

In [15]:
validated

,done instruction
dlg_id,
0,True
1,True
2,False
3,False
4,False
5,False


In [16]:
'Извлечение имени менеджера и компании'

'Извлечение имени менеджера и компании'

In [17]:
def get_name(x):
    s = str(x)
    
    name = re.search(r'(?<=Меня зовут)(\s\w+)', s)
    if name:
        return name.group(0)
    else: 
        name = re.search(r'меня(.*?)зовут', s)
        return name.group(1)

In [18]:
def get_company(x):
    s = str(x)
    
    company = re.search(r'компани(.*?)(\s\w+)', s)
    if company:
        return company.group(2)

In [19]:
searchfor_intro = ['зовут']
filt_intros = pd.Series(df['text'].str.lower().str.contains('|'.join(searchfor_intro)))

In [20]:
df['manager_intro_name'] = df.loc[filt_intros, ['text']].apply(get_name, axis=1)

In [21]:
df['manager_intro'] = df.loc[filt_intros, ['text']].apply(lambda x: True, axis=1)

In [22]:
df['company'] = df.loc[filt_intros, ['text']].apply(get_company, axis=1)

In [23]:
filt_notna_name = (df['manager_intro_name'].notna())
filt_general = (filt_greets_notna | filt_notna_name) & filt_manager

df = df[filt_general]

In [24]:
df = df.merge(validated, on='dlg_id', how='left')

In [25]:
df.to_excel('result.xlsx')